In [35]:
import re
ref_file = '/ldev/tensorflow/nmt2/nmt/data/wordlevel/test.40000.response'
embed_file = '/ldev/tensorflow/nmt2/word_embed/wiki.zh.vec'
eval_files = [
    '/ldev/tensorflow/nmt2/nmt/infer_test/RL0.test.txt',
    '/ldev/tensorflow/nmt2/nmt/beam_search/Subword2_10_test_f.inf.response',
    '/ldev/tensorflow/nmt2/nmt/beam_search/word_4W_10_test_f.inf.response',
    '/ldev/tensorflow/nmt2/nmt/beam_search/cnnEncDec_10_test_f.inf.response',

]
eval_subword = ['hybrid','bpe','','char2word',None]

# 统计所有用到的词
word_dict = set()

def encode(sentence, subword_option_1):
    sentence = sentence.strip('\n').lower()
    if subword_option_1 == 'bpe':
        sentence = re.sub("@@ ", "", sentence)
    if subword_option_1 == 'space':
        sentence = sentence.replace(" ", "")
        sentence = sentence.replace("<space>"," ")
    if subword_option_1 == 'char':
        sentence = sentence.replace("<space>", "")
        sentence = sentence.replace("@@", "")
        sentence = sentence.replace(" ","")
        sentence = " ".join(sentence)
    elif subword_option_1 == 'char2char':
        sentence = sentence.replace(" ", "")
        sentence = sentence.replace("@@", "")
        sentence = " ".join(sentence)
    elif subword_option_1 == 'char2word':
        sentence = sentence.replace(" ", "")
        sentence = sentence.replace("@@", " ")
        # sentence = " ".join(sentence)
    elif subword_option_1 == 'hybrid':
        sentence = sentence.replace(" @@ ", "")
        sentence = sentence.replace("@@ ", "")
        sentence = sentence.replace(" @@", "")
    elif subword_option_1 == 'hybrid2':
        sentence = sentence.replace(" ", "")
        sentence = sentence.replace("@@", " ")
    return sentence


for file,subword in zip(eval_files + [ref_file], eval_subword +[None]):
    with open(file, 'r', encoding = 'utf-8') as fin:
        for line in fin.readlines():
            encoded_line = encode(line,subword)
            for word in encoded_line.split(' '):
                word_dict.add(word.lower())

print('word nums : %d' % (len(word_dict)))

word nums : 31388


In [36]:
# with open(embed_file,encoding='ISO-8859-1') as f:
#     with open(embed_file+'.u8','w+',encoding='utf-8') as f2:
#         for line in f.readlines():
#             try:
#                 uft_str = line.encode("iso-8859-1").decode('utf-8') 
#                 f2.write(uft_str)
#             except :
#                 pass

In [37]:
# 加载embedding
import numpy as np
def read_vectors(path, topn):  # read top n word vectors, i.e. top is 10000
    lines_num, dim = 0, 0
    vectors = {}
    iw = []
    wi = {}
    with open(path,encoding='utf-8') as f:
        first_line = True
        for line in f:
            if first_line:
                first_line = False
                dim = int(line.strip().split()[1])
                continue
            lines_num += 1
            tokens = line.rstrip().split(' ')
            if tokens[0] in word_dict:
                vectors[tokens[0]] = np.asarray([float(x) for x in tokens[1:]])
                vectors[tokens[0]] = vectors[tokens[0]]
                iw.append(tokens[0])
            if topn != 0 and lines_num >= topn:
                break
    for i, w in enumerate(iw):
        wi[w] = i
    return vectors, iw, wi, dim

vectors = read_vectors(embed_file,0)
print(len(vectors[0]))

15198


In [34]:
# 计算一个文件的embedding
random_embedding = dict()
stop_words = set(['！','？','。','.','，'])
def get_embeddings(file,subword=None):
    embeddings = []
    valids = []
    count = 0
    with open(file, encoding='utf-8') as fin:
        for line in fin.readlines():
            encoded_line = encode(line,subword)
            tmp = np.zeros([300])
            count = 0
            for word in encoded_line.split(' '):
                if word in vectors[0]:
                    tmp += vectors[0][word]
                    count += 1
                else:
                    if word in random_embedding:
                        noisy = random_embedding[word]
                    else:  
                        noisy = np.random.normal(size=[300])
                        random_embedding[word] = noisy
                    tmp+=noisy
                    count += 1
            if count > 0:
                tmp = tmp / sum(np.sqrt(tmp*tmp))
                valids.append(1)
            else:
                valids.append(0)
            embeddings.append(tmp)
        return embeddings,valids
ref_embeddings  = get_embeddings(ref_file,None)
def distances(embedA,embedB,validA,validB):
    res = []
    
    for a,b,c,d in zip(embedA,embedB,validA,validB):
        dis = (sum(a*b)) / (np.sqrt(sum(a*a))* np.sqrt(sum(b*b))+1e-10)
        res.append(dis)
        
    return sum(res) / len(res)
embeddings = []
final_names = eval_files + [ref_file]
valids = []
for file,subword in zip(eval_files + [ref_file], eval_subword +[None]):
    print(file)
    embedding,valid = get_embeddings(file,subword)
    embeddings.append(embedding)
    valids.append(valid)

matrix = []
for i in range(len(embeddings)-1,len(embeddings)):
    row = []
    for j in range(0,len(embeddings)):
        diss = distances(embeddings[i],embeddings[j],valids[i],valids[j])
        row.append(diss)
    matrix.append(row)
print(matrix)

/ldev/tensorflow/nmt2/nmt/infer_test/RL0.test.txt
/ldev/tensorflow/nmt2/nmt/beam_search/Subword2_10_test_f.inf.response
/ldev/tensorflow/nmt2/nmt/beam_search/word_4W_10_test_f.inf.response
/ldev/tensorflow/nmt2/nmt/beam_search/cnnEncDec_10_test_f.inf.response
/ldev/tensorflow/nmt2/nmt/data/wordlevel/test.40000.response
[[0.001041069137744208, 0.0010710551323975907, 0.000797152680494999, 0.0005561805227645554, 0.9999999808640893]]


In [13]:
# # 计算一个文件的embedding Gready
# def get_embeddings(file,subword=None):
#     embeddings = []
#     valids = []
#     count = 0
#     with open(file, encoding='utf-8') as fin:
#         for line in fin.readlines():
#             encoded_line = encode(line,subword)
#             tmp = []
#             count = 0
#             for word in encoded_line.split(' '):
#                 if word in vectors[0]:
#                     tmp.append(vectors[0][word])
#                     count += 1
#                 else:
#                     noisy = np.random.uniform(size=[300])
#                     noisy = noisy/sum(np.sqrt(noisy*noisy))
#                     tmp.append(noisy)
#                     count += 1
#             if count > 0:
#                 valids.append(1)
#             else:
#                 valids.append(0)
#             embeddings.append(tmp)
#         return embeddings,valids
# ref_embeddings  = get_embeddings(ref_file,None)

# def distances(embedA,embedB,validA,validB):
#     res = []
    
#     for a,b,c,d in zip(embedA,embedB,validA,validB):
#         if c*d == 0:
#             print('error')
#         for r in a:
#             score1 = -1
#             for r1 in b:
#                 dis = (sum(r*r1)+1e-10) / (np.sqrt(sum(r*r))* np.sqrt(sum(r1*r1))+1e-10)
#                 score1 = max(dis,score1)
#         for r in b:
#             score2 = -1
#             for r1 in a:
#                 dis = (sum(r*r1)+1e-10) / (np.sqrt(sum(r*r))* np.sqrt(sum(r1*r1))+1e-10)
#                 score2 = max(dis,score2)
#         res.append(score1+score2)
        
#     return sum(res) / len(res) / 2
# embeddings = []
# final_names = eval_files + [ref_file]
# valids = []
# for file,subword in zip(eval_files + [ref_file], eval_subword +[None]):
#     print(file)
#     embedding,valid = get_embeddings(file,subword)
#     embeddings.append(embedding)
#     valids.append(valid)

# matrix = []
# for i in range(len(embeddings)-1,len(embeddings)):
#     row = []
#     for j in range(0,len(embeddings)):
#         diss = distances(embeddings[i],embeddings[j],valids[i],valids[j])
#         row.append(diss)
#     matrix.append(row)
# print(matrix)

/ldev/tensorflow/nmt2/nmt/infer_test/RL0.test.txt
/ldev/tensorflow/nmt2/nmt/beam_search/Subword2_10_test_f.inf.response
/ldev/tensorflow/nmt2/nmt/beam_search/word_4W_10_test_f.inf.response
/ldev/tensorflow/nmt2/nmt/beam_search/char_raw_10_test_f.inf.response
/ldev/tensorflow/nmt2/nmt/beam_search/cnnEncDec_10_test_f.inf.response


KeyboardInterrupt: 

/ldev/tensorflow/nmt2/nmt/infer_test/cn_albpe2.test.txt
/ldev/tensorflow/nmt2/nmt/infer_test/cn_rnn_hlat.test.txt
/ldev/tensorflow/nmt2/nmt/infer_test/bpe_lstm.test.txt
/ldev/tensorflow/nmt2/nmt/infer_test/cnnencdec.test.txt
/ldev/tensorflow/nmt2/nmt/infer_test/chinese_lstm.test.txt
/ldev/tensorflow/nmt2/nmt/data/wordlevel/test.40000.response
[[0.5710198406937087, 0.5525693729637218, 0.5578741953815899, 0.5658481167931326, 0.3565961653136164, 1.0]]


In [47]:
print(matrix)

[[0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 1.0, 0.9999999999999998]]
